<a href="https://colab.research.google.com/github/hssitja/PhD-Dissertation/blob/main/Ch2_Sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Setup
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

from google.colab import drive, auth
drive.mount('/content/drive')


import os
path = '/content/drive/My Drive/2 Essay Bayesian dinamic model for vaccine hesitancy/Data'
os.chdir(path)

project = 'phd-dissertation-343113' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

import pandas as pd
import numpy as np

with open('/content/drive/MyDrive/2 Essay Bayesian dinamic model for vaccine hesitancy/Python/Google developer/config') as f:
    api_key = f.readline()
    f.close

Mounted at /content/drive


## Reference SQL syntax from the original job
Use the ```jobs.query```
[method](https://cloud.google.com/bigquery/docs/reference/rest/v2/jobs/query) to
return the SQL syntax from the job. This can be copied from the output cell
below to edit the query now or in the future. Alternatively, you can use
[this link](https://console.cloud.google.com/bigquery?j=phd-dissertation-343113:US:bquxjob_2c4e677d_18627f112cb)
back to BigQuery to edit the query within the BigQuery user interface.

In [ ]:
# Running this code will display the query used to generate your previous job

job = client.get_job('bquxjob_8e8b9c8_1867a193ce4') # Job ID vaccination Porto Alegre
print(job.query)

WITH base_query AS (
  SELECT 
    id_paciente,
    idade_paciente,
    sexo_paciente,
    raca_cor_paciente,
    id_municipio_endereco_paciente,
    cep_endereco_paciente,
    data_aplicacao_vacina,
    nome_fabricante_vacina,
    dose_vacina,
    grupo_atendimento_vacina,
    ROW_NUMBER() OVER (PARTITION BY id_paciente ORDER BY data_aplicacao_vacina) AS dose_number
  FROM `basedosdados.br_ms_vacinacao_covid19.microdados`
  WHERE id_municipio_endereco_paciente IN ('1302603', '1500800', '1506807', '1600303', '2111300', '2303709', '2610707', '2611606', '3205309', '3304557', '3516200', '3530607', '4106902', '4115200', '4305108', '4314902', '5103403', '5201108', '5201405', '5300108')
), pivot_query AS (
  SELECT 
    id_paciente,
    MIN(idade_paciente) AS idade_paciente,
    MIN(sexo_paciente) AS sexo_paciente,
    MIN(raca_cor_paciente) AS raca_cor_paciente,
    MIN(id_municipio_endereco_paciente) AS id_municipio_endereco_paciente,
    MIN(cep_endereco_paciente) AS cep_endereco_paciente

In [ ]:
job2 = client.get_job('bquxjob_50765c8a_1867a237d5e') # Job ID votes by bairro
print(job2.query)

WITH municipios AS (
  SELECT id_municipio, nome, sigla_uf, nome_regiao
  FROM basedosdados.br_bd_diretorios_brasil.municipio
), bairros AS (
  SELECT id_municipio, secao, zona, bairro, cep
  FROM basedosdados.br_tse_eleicoes.perfil_eleitorado_local_votacao
  Where ano = 2022
), votacao AS(
  SELECT id_municipio, zona, secao,
  SUM(CASE WHEN ano = 2018 AND sigla_partido = 'PSL' THEN votos ELSE 0 END) AS votos_PSL_2018,
  SUM(CASE WHEN ano = 2018 AND sigla_partido = 'PT' THEN votos ELSE 0 END) AS votos_PT_2018,
  SUM(CASE WHEN ano = 2018 THEN votos ELSE 0 END) AS votos_tot_2018
  -- SUM(CASE WHEN ano = 2022 AND sigla_partido = 'PL' THEN votos ELSE 0 END) AS votos_PL_2022,
  -- SUM(CASE WHEN ano = 2022 AND sigla_partido = 'PT' THEN votos ELSE 0 END) AS votos_PT_2022,
  -- SUM(CASE WHEN ano = 2022 THEN votos ELSE 0 END) AS votos_tot_2022
  FROM `basedosdados.br_tse_eleicoes.resultados_candidato_secao`
  WHERE turno = 1 AND cargo = 'deputado federal'
  GROUP BY id_municipio, zona, secao
)


# Result set loaded from BigQuery job as a DataFrame
Query results are referenced from the Job ID ran from BigQuery and the query
does not need to be re-run to explore results. The ```to_dataframe```
[method](https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.job.QueryJob.html#google.cloud.bigquery.job.QueryJob.to_dataframe)
downloads the results to a Pandas DataFrame by using the BigQuery Storage API.

To edit query syntax, you can do so from the BigQuery SQL editor or in the
```Optional:``` sections below.

In [ ]:
# Running this code will read results from your previous job

job = client.get_job('bquxjob_8e8b9c8_1867a193ce4') # Job ID inserted based on the query results selected to explore
df = job.to_dataframe()
df.head(5)

,id_paciente,sexo_paciente,raca_cor_paciente,cep_endereco_paciente,id_municipio_endereco_paciente,idade_paciente,dose1,dose2,dose3,dose4,nome_fabricante_dose1,nome_fabricante_dose2,nome_fabricante_dose3,nome_fabricante_dose4,grupo_atendimento_dose1,diff_dose1_dose2,diff_dose2_dose3
0,dc434259fdfb193e7b326b49ebb5b60184deaefd8c7196...,M,1,91790,4314902,37,Dose Única,Reforço,None,None,JANSSEN,JANSSEN,None,None,201,157,<NA>
1,aa8cda473788ecc01ca7e11bdc12e188ecbdcf69c19275...,M,1,71686,5300108,25,Dose Única,Reforço,None,None,JANSSEN,JANSSEN,None,None,1501,145,<NA>
2,ddced77709fb3841c393e71f8295a90b1946353de563be...,M,99,21616,3304557,41,Dose Única,Reforço,None,None,JANSSEN,JANSSEN,None,None,201,189,<NA>
3,ec126e08f8ff3cac6fdeb56d10605651e7a88264d507ce...,F,2,91320,4314902,47,Dose Única,Dose Adicional,None,None,JANSSEN,JANSSEN,None,None,201,168,<NA>
4,ff9e4b082837216eb39b06d21e304e29a361a3f6477ab2...,M,3,72241,5300108,55,Dose Única,Reforço,None,None,JANSSEN,JANSSEN,None,None,201,169,<NA>


In [ ]:
job2 = client.get_job('bquxjob_50765c8a_1867a237d5e') # Job ID inserted based on the query results selected to explore
df2 = job2.to_dataframe()
df2.head(5)

,id_municipio,nome,sigla_uf,nome_regiao,bairro,votos_PSL_2018,votos_PT_2018,votos_tot_2018
0,3543402,Ribeirão Preto,SP,Sudeste,JARDIM CASTELO BRANCO,8664,742,34700
1,4103040,Boa Ventura de São Roque,PR,Sul,CENTRO,2042,3784,24042
2,2304400,Fortaleza,CE,Nordeste,BARRA DO CEARÁ,18176,51950,351040
3,2304400,Fortaleza,CE,Nordeste,BELA VISTA,9102,20530,194868
4,2203859,Floresta do Piauí,PI,Nordeste,CENTRO,0,0,0


## Show descriptive statistics using describe()
Use the ```pandas DataFrame.describe()```
[method](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html)
to generate descriptive statistics. Descriptive statistics include those that
summarize the central tendency, dispersion and shape of a dataset’s
distribution, excluding ```NaN``` values. You may also use other Python methods
to interact with your data.

In [ ]:
df2.describe()

,votos_PSL_2018,votos_PT_2018,votos_tot_2018
count,49224.000000,49224.000000,4.922400e+04
mean,2431.147448,2003.862872,2.066246e+04
std,6886.720885,4325.640316,3.800279e+04
min,0.000000,0.000000,0.000000e+00
25%,42.000000,168.000000,2.743500e+03
50%,356.000000,622.000000,8.362000e+03
75%,2040.000000,1960.000000,2.384650e+04
max,493920.000000,100080.000000,1.880226e+06


In [ ]:
# generating an array with the cep's

cep = df['cep_endereco_paciente'].unique()

**Loading CEP data and sorting by "bairro"**

In [ ]:
ceps = pd.read_csv('ceps-brasil.csv', sep='|')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
ceps.head(5)

,UF,Link UF,Municipio,Link Municipio,Bairro,Link Bairro,CEP,Localizacao,Link CEP
0,Acre,https://ceps.io/AC/related,Acrelandia,https://ceps.io/AC/Acrelandia/related,Centro,https://ceps.io/AC/Acrelandia/Centro/related,69945970,"Avenida Paraná, 296",https://ceps.io/AC/Acrelandia/Centro/Avenida-P...
1,Acre,https://ceps.io/AC/related,Assis Brasil,https://ceps.io/AC/Assis-Brasil/related,Centro,https://ceps.io/AC/Assis-Brasil/Centro/related,69935970,"Rua Dom Giocondo Maria Grotte, 230",https://ceps.io/AC/Assis-Brasil/Centro/Rua-Dom...
2,Acre,https://ceps.io/AC/related,Brasileia,https://ceps.io/AC/Brasileia/related,Centro,https://ceps.io/AC/Brasileia/Centro/related,69932970,"Avenida Prefeito Rolando Moreira, 170",https://ceps.io/AC/Brasileia/Centro/Avenida-Pr...
3,Acre,https://ceps.io/AC/related,Bujari,https://ceps.io/AC/Bujari/related,Centro,https://ceps.io/AC/Bujari/Centro/related,69926970,"Rua Expedito Pereira de Souza, 971",https://ceps.io/AC/Bujari/Centro/Rua-Expedito-...
4,Acre,https://ceps.io/AC/related,Campinas (Placido de Castro),https://ceps.io/AC/Campinas-Placido-de-Castro/...,Centro,https://ceps.io/AC/Campinas-Placido-de-Castro/...,69929970,Rua Kaxinawás,https://ceps.io/AC/Campinas-Placido-de-Castro/...


In [ ]:
# Limiting the cep columns to the first 5 digits and merging with df

df['cep_endereco_paciente'] = df['cep_endereco_paciente'].astype(str)
ceps['CEP'] = ceps['CEP'].astype(str)

bairros = ceps.groupby(ceps['CEP'].str[:5]).apply(lambda x: list(set(x['Bairro']))).reset_index()
bairros.columns = ['cep_endereco_paciente', 'bairros']

df_with_bairros = df.merge(bairros, on='cep_endereco_paciente', how='left')

In [ ]:
df_with_bairros.head(5)

,id_paciente,sexo_paciente,raca_cor_paciente,cep_endereco_paciente,id_municipio_endereco_paciente,idade_paciente,dose1,dose2,dose3,dose4,nome_fabricante_dose1,nome_fabricante_dose2,nome_fabricante_dose3,nome_fabricante_dose4,grupo_atendimento_dose1,diff_dose1_dose2,diff_dose2_dose3,bairros
0,dc434259fdfb193e7b326b49ebb5b60184deaefd8c7196...,M,1,91790,4314902,37,Dose Única,Reforço,None,None,JANSSEN,JANSSEN,None,None,201,157,<NA>,[Restinga]
1,aa8cda473788ecc01ca7e11bdc12e188ecbdcf69c19275...,M,1,71686,5300108,25,Dose Única,Reforço,None,None,JANSSEN,JANSSEN,None,None,1501,145,<NA>,"[Setor Habitacional Jardim Botânico, Setor de ..."
2,ddced77709fb3841c393e71f8295a90b1946353de563be...,M,99,21616,3304557,41,Dose Única,Reforço,None,None,JANSSEN,JANSSEN,None,None,201,189,<NA>,"[Marechal Hermes, Deodoro, Vila Militar]"
3,ec126e08f8ff3cac6fdeb56d10605651e7a88264d507ce...,F,2,91320,4314902,47,Dose Única,Dose Adicional,None,None,JANSSEN,JANSSEN,None,None,201,168,<NA>,"[Vila Ipiranga, Jardim Europa, Vila Jardim]"
4,ff9e4b082837216eb39b06d21e304e29a361a3f6477ab2...,M,3,72241,5300108,55,Dose Única,Reforço,None,None,JANSSEN,JANSSEN,None,None,201,169,<NA>,"[Cidade Industrial Satélite de São Paulo, Ceil..."


In [ ]:
df_with_bairros.bairros

0                                                  [Restinga]
1           [Setor Habitacional Jardim Botânico, Setor de ...
2                    [Marechal Hermes, Deodoro, Vila Militar]
3                 [Vila Ipiranga, Jardim Europa, Vila Jardim]
4           [Cidade Industrial Satélite de São Paulo, Ceil...
                                  ...                        
19291739                                          [Bacacheri]
19291740                     [Setor de Mansões de Sobradinho]
19291741          [Inhoaíba, Cosmos, Campo Grande, Paciência]
19291742    [Jardim Nogueira, Novo Pabussu, Padroeira, Vel...
19291743    [Vila Apaco, Santa Clara, Santa Efigênia, Cida...
Name: bairros, Length: 19291744, dtype: object

In [ ]:
# First, let's convert the columns to all caps
df_with_bairros['bairros'] = df_with_bairros['bairros'].apply(lambda x: [str(i).upper() for i in x] if isinstance(x, list) else str(x))

df2['bairro'] = df2['bairro'].apply(lambda x: x.upper())

# Create a dictionary mapping tuples of (id_municipio, bairro) to list of corresponding vote values
vote_dict = {}
for index, row in df2.iterrows():
    key = (row['id_municipio'], row['bairro'])
    vote_dict.setdefault(key, []).append([row['votos_PSL_2018'], row['votos_PT_2018'], row['votos_tot_2018']])

In [ ]:
# Loop through rows in df_with_bairros and get the average vote values for each bairro
for index, row in df_with_bairros.iterrows():
    bairros = row['bairros']
    id_municipio_endereco_paciente = row['id_municipio_endereco_paciente']
    vote_totals = [0] * 3
    count = 0
    for bairro in bairros:
        key = (id_municipio_endereco_paciente, bairro)
        if key in vote_dict:
            vote_totals = [sum(x) for x in zip(vote_totals, vote_dict[key][0])]
            count += 1
    if count > 0:
        vote_averages = [x / count for x in vote_totals]
        df_with_bairros.at[index, 'mean_votos_PSL_2018'] = vote_averages[0]
        df_with_bairros.at[index, 'mean_votos_PT_2018'] = vote_averages[1]
        df_with_bairros.at[index, 'mean_votos_tot_2018'] = vote_averages[2]


In [ ]:
df3 = pd.read_csv('df_features.csv')
df3['id_municipio'] = df3['id_municipio'].astype(str)
df3['income_avg'] = df3['income_avg'].replace('X', np.nan).astype(float)
df3['literacy'] = df3['literacy'].replace('X', np.nan).astype(float)
df3['families'] = df3['families'].replace('X', np.nan).astype(float)

In [ ]:
df3.describe()

,sample,income_avg,literacy,families
count,955.0,14016.000000,14016.000000,14016.000000
mean,1.0,2203.549654,93.123652,3.266129
std,0.0,1582.305082,6.523588,0.406278
min,1.0,299.340000,51.600000,1.390000
25%,1.0,1316.355000,91.300000,3.020000
50%,1.0,1799.585000,95.200000,3.230000
75%,1.0,2526.797500,97.500000,3.460000
max,1.0,35035.320000,100.000000,7.230000


In [ ]:
df2.describe()

,votos_PSL_2018,votos_PT_2018,votos_tot_2018
count,49224.000000,49224.000000,4.922400e+04
mean,2431.147448,2003.862872,2.066246e+04
std,6886.720885,4325.640316,3.800279e+04
min,0.000000,0.000000,0.000000e+00
25%,42.000000,168.000000,2.743500e+03
50%,356.000000,622.000000,8.362000e+03
75%,2040.000000,1960.000000,2.384650e+04
max,493920.000000,100080.000000,1.880226e+06


In [ ]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14113 entries, 0 to 14112
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id_municipio  14113 non-null  object 
 1   bairro        14113 non-null  object 
 2   nome          14113 non-null  object 
 3   nome_regiao   14113 non-null  object 
 4   sample        955 non-null    float64
 5   income_avg    14016 non-null  float64
 6   literacy      14016 non-null  float64
 7   families      14016 non-null  float64
dtypes: float64(4), object(4)
memory usage: 882.2+ KB


In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49224 entries, 0 to 49223
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id_municipio    49224 non-null  object
 1   nome            49224 non-null  object
 2   sigla_uf        49224 non-null  object
 3   nome_regiao     49224 non-null  object
 4   bairro          49224 non-null  object
 5   votos_PSL_2018  49224 non-null  Int64 
 6   votos_PT_2018   49224 non-null  Int64 
 7   votos_tot_2018  49224 non-null  Int64 
dtypes: Int64(3), object(5)
memory usage: 3.1+ MB


In [ ]:
# Create a dictionary mapping tuples of (id_municipio, bairro) to list of corresponding feature values
features_dict = {}
for index, row in df3.iterrows():
    key = (row['id_municipio'], row['bairro'])
    features_dict.setdefault(key, []).append([row['income_avg'], row['literacy'], row['families']])

# Loop through rows in df_with_bairros and get the average feature values for each bairro
for index, row in df_with_bairros.iterrows():
    bairros = row['bairros']
    id_municipio_endereco_paciente = row['id_municipio_endereco_paciente']
    feature_totals = [0] * 3
    count = 0
    for bairro in bairros:
        key = (id_municipio_endereco_paciente, bairro)
        if key in features_dict:
            feature_totals = [sum(x) for x in zip(feature_totals, features_dict[key][0])]
            count += 1
    if count > 0:
        feature_averages = [x / count for x in feature_totals]
        df_with_bairros.at[index, 'mean_income_avg'] = feature_averages[0]
        df_with_bairros.at[index, 'mean_literacy'] = feature_averages[1]
        df_with_bairros.at[index, 'mean_families'] = feature_averages[2]


In [ ]:
df_with_bairros.head(5)

,id_paciente,sexo_paciente,raca_cor_paciente,cep_endereco_paciente,id_municipio_endereco_paciente,idade_paciente,dose1,dose2,dose3,dose4,...,grupo_atendimento_dose1,diff_dose1_dose2,diff_dose2_dose3,bairros,mean_votos_PSL_2018,mean_votos_PT_2018,mean_votos_tot_2018,mean_income_avg,mean_literacy,mean_families
0,dc434259fdfb193e7b326b49ebb5b60184deaefd8c7196...,M,1,91790,4314902,37,Dose Única,Reforço,None,None,...,201,157,<NA>,[RESTINGA],16700.0,44026.000000,284284.000000,1573.44,96.000000,3.440000
1,aa8cda473788ecc01ca7e11bdc12e188ecbdcf69c19275...,M,1,71686,5300108,25,Dose Única,Reforço,None,None,...,1501,145,<NA>,"[SETOR HABITACIONAL JARDIM BOTÂNICO, SETOR DE ...",NaN,NaN,NaN,NaN,NaN,NaN
2,ddced77709fb3841c393e71f8295a90b1946353de563be...,M,99,21616,3304557,41,Dose Única,Reforço,None,None,...,201,189,<NA>,"[MARECHAL HERMES, DEODORO, VILA MILITAR]",19506.0,1756.666667,71928.666667,2222.86,97.866667,3.153333
3,ec126e08f8ff3cac6fdeb56d10605651e7a88264d507ce...,F,2,91320,4314902,47,Dose Única,Dose Adicional,None,None,...,201,168,<NA>,"[VILA IPIRANGA, JARDIM EUROPA, VILA JARDIM]",11348.0,12767.000000,121568.000000,3546.25,98.700000,2.685000
4,ff9e4b082837216eb39b06d21e304e29a361a3f6477ab2...,M,3,72241,5300108,55,Dose Única,Reforço,None,None,...,201,169,<NA>,"[CIDADE INDUSTRIAL SATÉLITE DE SÃO PAULO, CEIL...",NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Save df_with_bairros as a CSV file named df_city.csv

city = 'Sample'

df_with_bairros.to_csv(f'df_{city}.csv', index=False)

In [ ]:
# Convert IntegerArray to numpy array
df_with_bairros['mean_votos_PSL_2018'] = df_with_bairros['mean_votos_PSL_2018'].to_numpy()
df_with_bairros['mean_votos_PT_2018'] = df_with_bairros['mean_votos_PT_2018'].to_numpy()
df_with_bairros['mean_votos_tot_2018'] = df_with_bairros['mean_votos_tot_2018'].to_numpy()
# df_with_bairros['mean_votos_PL_2022'] = df_with_bairros['mean_votos_PL_2022'].to_numpy()
# df_with_bairros['mean_votos_PT_2022'] = df_with_bairros['mean_votos_PT_2022'].to_numpy()
# df_with_bairros['mean_votos_tot_2022'] = df_with_bairros['mean_votos_tot_2022'].to_numpy()
df_with_bairros['mean_income_avg'] = df_with_bairros['mean_income_avg'].to_numpy()
df_with_bairros['mean_literacy'] = df_with_bairros['mean_literacy'].to_numpy()
df_with_bairros['mean_families'] = df_with_bairros['mean_families'].to_numpy()

# Convert bairros to string representation of the lists
df_with_bairros['bairros'] = df_with_bairros['bairros'].apply(lambda x: str(x) if isinstance(x, list) else x)

# Convert any np.integer values to int to avoid errors
int_cols = df_with_bairros.select_dtypes(include=[np.integer]).columns

# Loop over the integer columns and convert them to NumPy arrays
for col in int_cols:
    df_with_bairros[col] = df_with_bairros[col].to_numpy()

# Loop over the integer columns and cast non-NaN values to int64
# for col in int_cols:
#     nan_mask = df_with_bairros[col].isna()
#     df_with_bairros.loc[~nan_mask, col] = df_with_bairros.loc[~nan_mask, col].astype('int64')
for col in int_cols:
    df_with_bairros[col] = df_with_bairros[col].fillna(0).astype('int64')

# date_cols = ['data_aplicacao_dose1', 'data_aplicacao_dose2', 'data_aplicacao_dose3', 'data_aplicacao_dose4', 'data_aplicacao_dose5']
# for col in date_cols:
#     df_with_bairros[col] = df_with_bairros[col].apply(lambda x: pd.Timestamp(x))

In [ ]:
df_with_bairros.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19291744 entries, 0 to 19291743
Data columns (total 24 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   id_paciente                     object 
 1   sexo_paciente                   object 
 2   raca_cor_paciente               object 
 3   cep_endereco_paciente           object 
 4   id_municipio_endereco_paciente  object 
 5   idade_paciente                  int64  
 6   dose1                           object 
 7   dose2                           object 
 8   dose3                           object 
 9   dose4                           object 
 10  nome_fabricante_dose1           object 
 11  nome_fabricante_dose2           object 
 12  nome_fabricante_dose3           object 
 13  nome_fabricante_dose4           object 
 14  grupo_atendimento_dose1         object 
 15  diff_dose1_dose2                int64  
 16  diff_dose2_dose3                int64  
 17  bairros                  

In [ ]:
!pip install vaex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 KB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.2/58.2 KB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.0/239.0 KB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.2/111.2 KB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 KB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 94.

In [ ]:
# Save to hdf5
import vaex

vaex_df = vaex.from_pandas(df_with_bairros, copy_index=False)
vaex_df.export_hdf5(f'df_{city}.hdf5')